In [1]:
import pyodbc
import pandas as pd
import datetime

pd.set_option('display.width', 1000)

connection_results_db = pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};'
                                       'Dbq=S:\\Audit\\Automated testing\\Access controls\\Results\\results_clean.accdb;')
cur = connection_results_db.cursor()
print('dONE')
# To get update the "results_transposed" table we check for where the date tested does not exist in the 
# "result_detail" table.
dates_in_result_detail = pd.read_sql(sql="""select distinct DAY_TESTING_PERFORMED from result_detail""", 
                         con=connection_results_db)['DAY_TESTING_PERFORMED'].tolist()
print('dONE')
dates_in_results_transposed = pd.read_sql(sql="""select distinct DAY_TESTING_PERFORMED from results_transposed""", 
                         con=connection_results_db)['DAY_TESTING_PERFORMED'].tolist()
print('dONE')
dates_to_update = [d for d in dates_in_result_detail if d not in dates_in_results_transposed]
print('dONE')
if len(dates_to_update) == 1:
    dates_to_update = "('" + dates_to_update[0] + "')"
else:
    dates_to_update = tuple(dates_to_update)

print(dates_to_update)

df_results = pd.read_sql(sql="""select * from result_detail where DAY_TESTING_PERFORMED in {0}""".format(dates_to_update), 
                        con=connection_results_db)
print('dONE')

dONE
dONE
dONE
dONE
('2018-04-09')
dONE


In [2]:
melt_columns = [col for col in df_results.columns if col not in ['RESULT_DORMANT_ACCOUNT', 
                                                                 'RESULT_TERM_STAFF_ACTIVE', 
                                                                 'RESULT_GENERIC_ACCOUNT_SCORE', 
                                                                 'RESULT_INACTIVE_ACCOUNT',
                                                                 'RESULT_DATA_ISSUE_00', 
                                                                 'RESULT_DATA_ISSUE_01', 
                                                                 'RESULT_DUPLICATE_USER_ID', 
                                                                 'RESULT_DUPLICATE_USER_NAME', 
                                                                 'RESULT_ACTIVE_UNDEFINED_ENTITY', 
                                                                 'RESULT_IS_SUPERUSER']]
df_new = df_results.melt(id_vars =melt_columns, 
                var_name='TEST_PERFORMED', 
                value_name='RESULT').sort_values('STAFF_EMPLOYEECODE').reset_index(drop=True)

In [3]:
new_names_mapping = {'RESULT_DORMANT_ACCOUNT': 'Dormant accounts', 
                     'RESULT_TERM_STAFF_ACTIVE': 'Terminated staff with active accounts',
                     'RESULT_GENERIC_ACCOUNT_SCORE': 'Generic accounts',
                     'RESULT_INACTIVE_ACCOUNT': 'Inactive accounts',
                     'RESULT_DATA_ISSUE_00': 'Data issue I',
                     'RESULT_DATA_ISSUE_01': 'Data issue II',
                     'RESULT_DUPLICATE_USER_ID': 'Duplicate user id',
                     'RESULT_DUPLICATE_USER_NAME': 'Duplicate user name',  
                     'RESULT_ACTIVE_UNDEFINED_ENTITY': 'Active but undefined', 
                     'RESULT_IS_SUPERUSER': 'Superusers'}

df_new = df_new.replace({'TEST_PERFORMED': new_names_mapping})
df_new['TEST_PERFORMED'].unique()

array(['Terminated staff with active accounts', 'Duplicate user id',
       'Active but undefined', 'Data issue II', 'Inactive accounts',
       'Data issue I', 'Dormant accounts', 'Superusers',
       'Generic accounts', 'Duplicate user name'], dtype=object)

In [4]:
df_new_export_columns = df_new[['DAY_TESTING_PERFORMED', 
                                'USER_SOURCE_APPLICATION',
                                'USER_ID', 
                                'TEST_PERFORMED', 
                                'RESULT', 
                                'RESULT_IS_EMPLOYEE']].sort_values('DAY_TESTING_PERFORMED', ascending=True).reset_index(drop=True)

df_new_export_columns = df_new_export_columns.astype(str)

In [5]:
import progressbar 
import time
bar = progressbar.ProgressBar(max_value=len(df_new_export_columns))

for date in df_new_export_columns['DAY_TESTING_PERFORMED'].unique().tolist():
    df_new_export_columns_date = df_new_export_columns[df_new_export_columns['DAY_TESTING_PERFORMED'] ==  date]

    for index, row, in df_new_export_columns_date.iterrows():

        cur.execute("""INSERT INTO results_transposed values {0}""".format(tuple(row)))
        cur.commit()
        bar.update(index)


 99% (50403 of 50420) |################################################# | Elapsed Time: 0:05:45 ETA: 0:00:00